In [109]:
import time
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from scipy.ndimage import gaussian_filter, median_filter
import plotly.express as px

## PREPROCESSING

In [110]:
X = pd.read_csv("src/noisy_images.csv")

In [111]:
X.shape

(20, 784)

In [112]:
X.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,-176.321245,-52.743850,143.604939,-53.567749,80.289381,20.686379,-197.423519,-229.401206,-221.900009,64.194187,...,34.407070,1.950735,-25.095565,133.684095,-21.664094,-94.305438,-55.987821,-89.929231,40.394774,-214.754480
1,-158.421239,16.371695,62.810879,263.533916,-193.920320,-25.366668,107.062706,125.403427,83.536343,-55.713640,...,-5.419961,-123.584030,20.240434,-25.699206,-128.545930,52.525885,-54.214887,-133.842624,-30.141215,210.408665
2,-290.343750,81.586500,12.615232,146.567851,111.233602,-188.989259,-101.464605,-107.015195,-13.069827,-245.921093,...,-139.909318,-85.214133,167.495617,62.402411,-144.402970,152.263950,-4.687051,-59.270131,-93.193600,188.229794
3,-208.840590,136.190431,38.552191,-67.825346,24.316303,176.103673,31.581298,-163.582673,29.777077,-110.969396,...,-131.385192,40.329733,-10.111639,163.497435,41.010287,-21.408008,328.274235,-15.341672,121.570863,151.757537
4,-328.876288,-42.862900,174.651874,-228.833439,71.909654,-97.206392,48.048853,-34.071313,3.820465,137.807730,...,-135.713199,-71.396796,155.237981,-141.860908,155.657335,166.609760,-52.911774,267.150703,-36.749672,131.913772


## DISPLAY

In [113]:
def show_images(data: pd.DataFrame, title="Result", col_wrap=10):
    side_length = int(np.sqrt(data.shape[1]))
    fig = px.imshow(data.values.reshape(-1, side_length, side_length),
                    binary_string=True,
                    facet_col=0,
                    facet_col_wrap=col_wrap,
                    title=title)
    fig.show()

In [114]:
col_wrap = 10
show_images(X, "Original", col_wrap=col_wrap)

## Let's denoise with sklearn PCA and scipy filters !

In [115]:
def apply_filters(images, side_length: int, sigma: int = None, size: int = None):
    rows = []
    for image in images:
        image = image.reshape(side_length, side_length)
        if size:
            image = median_filter(image, size=size)
        if sigma:
            image = gaussian_filter(image, sigma=sigma)
        rows.append(image.flatten())

    return pd.DataFrame(rows)

In [116]:
def denoise_sklearn(images: pd.DataFrame,
            pca_components: float = 20,
            gaussian_sigma: int = 1,
            median_size: int = 1):

    side_length = int(np.sqrt(images.shape[1]))

    sk_pca = PCA(pca_components)
    images = sk_pca.fit_transform(images.values)
    images = sk_pca.inverse_transform(images)

    images = apply_filters(images,
                        side_length,
                        sigma=gaussian_sigma,
                        size=median_size)

    return pd.DataFrame(images), sk_pca

In [133]:
start = time.time()
sklearn_result, sk_pca = denoise_sklearn(X, pca_components=.7, gaussian_sigma=.8, median_size=1)
show_images(sklearn_result, "SKlearn PCA + scipy filters", col_wrap=col_wrap)
sk_time = time.time() - start

## Let's create our own PCA !

In [118]:
class MyPCA:
    def __init__(self, n_components: int = None) -> None:
        self._components = n_components

    def normalize_x(self, X):
        mean = np.mean(X, axis=0)
        std_dev = np.std(X, axis=0)

        X = (X - mean) / std_dev

        return X, mean, std_dev

    def fit_transform(self, X):
        if self._components is None:
            self._components = X.shape[1]

        # 1. Normalize X
        X, self._X_mean, self._X_std_dev = self.normalize_x(X)

        # 2. Calculate covariance matrix
        cov_matrix = np.cov(X, rowvar=False)

        # 3. Calculate eigen-vectors and eigen-values
        self._eig_vals, self._eig_vecs = np.linalg.eig(cov_matrix)

        # 4. Deduct PCA
        indices = np.argsort(self._eig_vals)[::-1]

        sort_eig_vecs = self._eig_vecs[:, indices]
        self._sort_eig_vals = self._eig_vals[indices]

        self._sel_eig_vecs = sort_eig_vecs[:, :self._components]

        return np.dot(X, self._sel_eig_vecs)

    def explained_variance_ratio_(self):
        return np.real(self._sort_eig_vals / np.sum(self._sort_eig_vals))[:self._components]

    def inverse_transform(self, X_pca):
        dot_product = np.dot(X_pca, self._sel_eig_vecs.T)
        return np.real((dot_product * self._X_std_dev) + self._X_mean)  # avoid complex issues

In [119]:
def denoise(images: pd.DataFrame,
            pca_components: float = None,
            gaussian_strength: int = None,
            median_strength: int = None):

    side_length = int(np.sqrt(images.shape[1]))

    my_pca = MyPCA(n_components=pca_components)
    images = my_pca.fit_transform(images.values)
    images = my_pca.inverse_transform(images)

    images = apply_filters(images,
                           side_length,
                           sigma=gaussian_strength,
                           size=median_strength)

    return pd.DataFrame(images), my_pca

In [120]:
start = time.time()
my_pca_result, my_pca = denoise(X, pca_components=15, gaussian_strength=None, median_strength=None)
show_images(my_pca_result, "My PCA + scipy filters", col_wrap=col_wrap)
my_time = time.time() - start

In [121]:
# Guillaume's PCA

def guillaume_pca(df):
    covariance_matrix = np.cov(df, rowvar=False)
    _, eigenvectors = np.linalg.eig(covariance_matrix)
    return df.dot(eigenvectors)

## ANALYSING RESULTS

In [122]:
print(f"SKLearn PCA -> {sk_pca.explained_variance_ratio_.sum()} in {sk_time} seconds")
print(f"Manual PCA -> {my_pca.explained_variance_ratio_().sum()} in {my_time} seconds")

SKLearn PCA -> 0.849988424446153 in 0.11943364143371582 seconds
Manual PCA -> 0.8403438394894851 in 0.8397681713104248 seconds


## EXPORT / REIMPORT

In [123]:
my_pca_result.to_csv("output/denoised_images.csv", index=False)

In [124]:
result_df = pd.read_csv("output/denoised_images.csv")

fig_noise = px.imshow(result_df.values.reshape(-1, 28, 28),
                      binary_string=True,
                      facet_col=0,
                      facet_col_wrap=10,
                      title="Result")
fig_noise.show()